In [1]:
#loading packages
%matplotlib notebook
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import glob

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
nx = 9
ny = 6
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
#print(objp[-1], objp[-nx], objp[0], objp[nx-1])
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def corners_unwarp(img, nx, ny, mtx, dist, show_plots = False):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    if show_plots:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
        ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=16)
        ax2.imshow(undist)
        ax2.set_title('Undistorted Image', fontsize=16)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    # Return the resulting image and matrix
    return undist

In [3]:
imgs = glob.glob('camera_cal/calibration*.jpg')

for i in imgs:
    img = cv2.imread(i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        #corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        c_img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
        #f.tight_layout()
        print('Image: ' + i)
        #ax1.imshow(cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB))
        #ax1.set_title('Original Image', fontsize=16)
        #ax2.imshow(c_img)
        #ax2.set_title('Chess Board Corners', fontsize=16)
        #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#cv2.destroyAllWindows()

Image: camera_cal\calibration10.jpg
Image: camera_cal\calibration11.jpg
Image: camera_cal\calibration12.jpg
Image: camera_cal\calibration13.jpg
Image: camera_cal\calibration14.jpg
Image: camera_cal\calibration15.jpg
Image: camera_cal\calibration16.jpg
Image: camera_cal\calibration17.jpg
Image: camera_cal\calibration18.jpg
Image: camera_cal\calibration19.jpg
Image: camera_cal\calibration2.jpg
Image: camera_cal\calibration20.jpg
Image: camera_cal\calibration3.jpg
Image: camera_cal\calibration6.jpg
Image: camera_cal\calibration7.jpg
Image: camera_cal\calibration8.jpg
Image: camera_cal\calibration9.jpg


In [4]:
def process_image(image, show_plot=False):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    #print('Received image : ',image)
    if image is None:
        return image
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    
    # you should return the final output (image where lines are drawn on lanes)
    
    return corners_unwarp(image, nx, ny, mtx, dist, show_plot)

In [5]:
def warper(img, show_plot = False):
    p_img = process_image(img)
    src = np.float32([[490, 482],[810, 482],
                      [1250, 720],[40, 720]])
    dst = np.float32([[0, 0], [1280, 0], 
                     [1250, 720],[40, 720]])
    # Compute and apply perpective transform
    img_size = (p_img.shape[1], p_img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(p_img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    if show_plot:
        # Plot the result
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=16)
        ax2.imshow(warped)
        ax2.set_title('Perspective Image', fontsize=16)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    return warped, Minv, p_img

In [6]:
def plot_imgs(org,trans,org_title = "Original Image", trans_title="Transformed Image"):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
    f.tight_layout()
    ax1.imshow(org)
    ax1.set_title(org_title, fontsize=16)
    ax2.imshow(trans)
    ax2.set_title(trans_title, fontsize=16)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [7]:
# Read in an image and grayscale it
image = mpimg.imread('signs_vehicles_xygrad.png')
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return mag_binary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    # Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    return dir_binary

# Choose a Sobel kernel size
ksize = 9 # Choose a larger odd number to smooth gradient measurements

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(0, 255))
grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(0, 255))
mag_binary = mag_thresh(image, sobel_kernel=3, mag_thresh=(30, 100))
dir_binary = dir_threshold(image, sobel_kernel=15, thresh=(0.7, 1.3))

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(mag_binary, cmap='gray')
ax2.set_title('Thresholded Grad. Dir.', fontsize=20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

<IPython.core.display.Javascript object>

In [8]:
image = mpimg.imread('signs_vehicles_xygrad.png')
# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(130, 255), sx_thresh=(20, 100),show_plot = False):
    img = np.copy(img)
    img, M, undist = warper(img, show_plot)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    if show_plot:
        plot_imgs(img, hls,"Warped Image","Converted to HLS color space")
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    if show_plot:
        plot_imgs(hls, scaled_sobel,"HLS color space","Applied x derivative")
    #print(scaled_sobel)
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    if show_plot:
        plot_imgs(scaled_sobel, sxbinary,"Applied x derivative","Applied x gradient")
    
    # Threshold color channel
    s_channel = s_channel*255
    s_binary = np.zeros_like(s_channel)
    if show_plot:
        print(np.max(s_channel), np.min(s_channel))
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    if show_plot:
        plot_imgs(s_channel, s_binary,"S channel","Applied Threshold color channel")
    
    comb_channel = np.zeros_like(s_channel)
    comb_channel [(s_binary == 1)|(sxbinary == 1)] = 1
    if show_plot:
        plot_imgs(img, comb_channel,"Perspective image","Combined channel output")
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    #plot_imgs(s_binary, color_binary,"Applied Threshold color channel","color binary")
    return comb_channel,M, undist
    
result, M,undist = pipeline(image, show_plot = True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

255.0 0.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
def sliding_window(binary_warped, show_plot = False):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    #print(window_height)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    #print(out_img)
    if show_plot:
        plot_imgs(binary_warped,out_img,"Combined Channel Image", "Found Lines")
    return left_fit, right_fit, left_lane_inds, right_lane_inds
sliding_window(result, True)

<IPython.core.display.Javascript object>

(array([  1.87815441e-05,  -4.63743650e-02,   1.24684070e+02]),
 array([ -1.19608040e-04,   2.86968274e-01,   8.50604249e+02]),
 array([19183, 19184, 19185, ...,  4482,  4483,  4484], dtype=int64),
 array([19207, 19208, 19209, ...,  4521,  4522,  4523], dtype=int64))

In [10]:
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
def continue_find_lane(binary_warped,left_fit,right_fit,ploty):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fit, right_fit, left_lane_inds, right_lane_inds

In [11]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    def add_fit(self, fit, inds):
        # add a found fit to the line, up to n
        if fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(fit-self.best_fit)
            if (self.diffs[0] > 0.01 or self.diffs[1] > 1.0 or self.diffs[2] > 100.) and len(self.current_fit) > 0:
                # wrong fit
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(inds)
                self.current_fit.append(fit)
                if len(self.current_fit) > 5:
                    # throw out old fits, keep newest n
                    self.current_fit = self.current_fit[len(self.current_fit)-5:]
                self.best_fit = np.average(self.current_fit, axis=0)
        
        else:
            self.detected = False
            if len(self.current_fit) > 0:
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
            if len(self.current_fit) > 0:
                self.best_fit = np.average(self.current_fit, axis=0)
                
        

In [12]:
def calc_curve_radius(comb_channel, l_lane_inds,r_lane_inds):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 3.048/100 # meters per pixel in y dimension, lane line is 10 ft = 3.048 meters
    xm_per_pix = 3.7/378 # meters per pixel in x dimension, lane width is 12 ft = 3.7 meters
    left_curverad, right_curverad = (0, 0)
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    s = comb_channel.shape[0]
    ploty = np.linspace(0, s-1, s)
    y_eval = np.max(ploty)
  
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = comb_channel.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Again, extract left and right line pixel positions
    leftx = nonzerox[l_lane_inds]
    lefty = nonzeroy[l_lane_inds] 
    rightx = nonzerox[r_lane_inds]
    righty = nonzeroy[r_lane_inds]

    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    #print(left_curverad, right_curverad)
    return (left_curverad + right_curverad)/2
    # Example values: 1926.74 1908.48

In [13]:

#plot lines (green color) in a trapezoidal shape on our lane
def plot_line(image,comb_channel, left_fit, right_fit, ploty, M):
    nimg = np.copy(image)
    # Create an image to draw the lines on
    if left_fit is None or right_fit is None:
        return image
    warp_zero = np.zeros_like(comb_channel).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the comb_channel blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(255,0,255), thickness=15)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(0,255,255), thickness=15)
    #print(M)
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M, (comb_channel.shape[1], comb_channel.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(nimg, 1, newwarp, 0.3, 0)
    return result

In [14]:
def plot_lines_for_img(t_img, show_output = False):
    comb_channel, Minv, undist = pipeline(t_img, show_plot = False)
    thres, margin = 350,100
    ploty = np.linspace(0, comb_channel.shape[0]-1, num=comb_channel.shape[0])# to cover same y-range as image    
    # if both left and right lines were detected last frame, use polyfit_using_prev_fit, otherwise use sliding window
    if not l_line.detected or not r_line.detected:
        left_fit, right_fit, l_lane_inds, r_lane_inds = sliding_window(comb_channel, False)
    else:
        left_fit, right_fit, l_lane_inds, r_lane_inds = continue_find_lane(comb_channel, l_line.best_fit, r_line.best_fit,ploty)
        
    
    plotted_line = plot_line(t_img, comb_channel, left_fit, right_fit, ploty, M)
    #plotted_line = draw_lane(t_img, comb_channel, left_fit, right_fit, M)
    radius = calc_curve_radius(comb_channel, l_lane_inds,r_lane_inds)
    font = cv2.FONT_HERSHEY_SIMPLEX
    msg = 'Curve radius: ' + str(radius) + 'meters'
    cv2.putText(plotted_line, msg,(40,100), font, 2,(255,255,255),2,cv2.LINE_AA)
    # invalidate both fits if the difference in their x-intercepts isn't around 350 px (+/- 100 px)
    if left_fit is not None and right_fit is not None:
        # calculate x-intercept (bottom of image, x=image_height) for fits
        s = t_img.shape[0]
        left_fit_x_int = left_fit[0]*s**2 + left_fit[1]*s + left_fit[2]
        right_fit_x_int = right_fit[0]*s**2 + right_fit[1]*s + right_fit[2]
        x_int_diff = abs(right_fit_x_int-left_fit_x_int)
        if abs(thres - x_int_diff) > margin:
            left_fit = None
            right_fit = None
            
    l_line.add_fit(left_fit, l_lane_inds)
    r_line.add_fit(right_fit, r_lane_inds)
    if show_output:
        plot_imgs(t_img, plotted_line,"Original Image","Found lane and Curvature")
    return plotted_line


In [15]:
l_line = Line()
r_line = Line()
test_imgs = glob.glob('test_images/test*.jpg')
for t_name in test_imgs[0:6]:
    t_img = cv2.imread(t_name)
    #pipeline(t_img, show_plot = True)
    plot_lines_for_img(t_img, True)   


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
l_line = Line()
r_line = Line()
# If the input is the camera, pass 0 instead of the video file name
output_video = 'videos_output/project_video.mp4'

clip = VideoFileClip("project_video.mp4") 
output_clip = clip.fl_image(plot_lines_for_img) #NOTE: this function expects color images!!
%time output_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video videos_output/project_video.mp4
[MoviePy] Writing video videos_output/project_video.mp4


 32%|██████████████████████████████████████████████████████████████████████▏                                                                                                                                                       | 399/1261 [07:33<12:20,  1.16it/s]

MemoryError: 

In [17]:
l_line = Line()
r_line = Line()
# If the input is the camera, pass 0 instead of the video file name
output_video = 'videos_output/challenge_video.mp4'

clip = VideoFileClip("challenge_video.mp4") 
output_clip = clip.fl_image(plot_lines_for_img) #NOTE: this function expects color images!!
%time output_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video videos_output/challenge_video.mp4
[MoviePy] Writing video videos_output/challenge_video.mp4



  6%|█████████████▊                                                                                                                                                                                                                  | 30/485 [00:24<07:34,  1.00it/s]


 13%|████████████████████████████▏                                                                                                                                                                                                   | 61/485 [00:52<05:26,  1.30it/s]


 14%|██████████████████████████████▍                                                                                                                                                                                                 | 66/485 [00:56<05:21,  1.30it/s]


MemoryError: 

In [18]:
l_line = Line()
r_line = Line()
# If the input is the camera, pass 0 instead of the video file name
output_video = 'videos_output/harder_challenge_video.mp4'

clip = VideoFileClip("harder_challenge_video.mp4") 
output_clip = clip.fl_image(plot_lines_for_img) #NOTE: this function expects color images!!
%time output_clip.write_videofile(output_video, audio=False)

error: ..\..\..\modules\core\src\alloc.cpp:52: error: (-4) Failed to allocate 7372800 bytes in function cv::OutOfMemoryError
